In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append("../library")
import RecSys as rs
#patch = "" # For Google CoLab
path = "../data/"

In [59]:
chefmozacccepts = pd.read_csv(path+"chefmozaccepts.csv")
chefmozcuisine = pd.read_csv(path+"chefmozcuisine.csv")
chefmozhours4 = pd.read_csv(path+"chefmozhours4.csv")
chefmozparking = pd.read_csv(path+"chefmozparking.csv")
geoplaces2 = pd.read_csv(path+"geoplaces2.csv")
rating_final = pd.read_csv(path+"rating_final.csv")
usercuisine = pd.read_csv(path+"usercuisine.csv")
userpayment = pd.read_csv(path+"userpayment.csv")
userprofile = pd.read_csv(path+"userprofile.csv")
# This will contain the list of Datas that we will 
# use for the Recommendation System
listOfData_r = [rating_final]

In [60]:
listOfData = [chefmozacccepts,chefmozcuisine,chefmozhours4,
              chefmozparking,geoplaces2,rating_final,
              usercuisine,userpayment,userprofile]

In [61]:
### We have to do the secondview clean

In [62]:
rs.replaceID(realID=132706,falseID=132584,ID='placeID',data=listOfData_r)

In [63]:
###############
# Those will be the data that we will use for the model and app
###############
userProfile_r = userprofile[['userID', 'smoker', 'drink_level','dress_preference', 
                             'ambience', 'transport', 'marital_status','hijos', 
                             'religion', 'activity']]
usercuisine_list = usercuisine.groupby('userID').apply(lambda x: list(x['Rcuisine']))
usercuisine_list = pd.DataFrame({'userID':usercuisine_list.index,'Rcuisine':usercuisine_list.values})
userProfile_r = userProfile_r.merge(usercuisine_list, how = 'left', on = 'userID')
userpayment_list= userpayment.groupby('userID').apply(lambda x: list(x['Upayment']))
userpayment_list = pd.DataFrame({'userID':userpayment_list.index,'Upayment':userpayment_list.values})
userProfile_r = userProfile_r.merge(userpayment_list, how = 'left', on = 'userID')

userGeo_r = userprofile[['userID', 'latitude', 'longitude']]

userChefmozRelation_r = rating_final
userChefmozRelation_r = userChefmozRelation_r.merge(userpayment, how='left', on='userID')
userChefmozRelation_r = userChefmozRelation_r.merge(usercuisine, how='left', on='userID')

chefmozProfile_r = geoplaces2[['placeID', 'alcohol', 'smoking_area','dress_code',
                               'accessibility', 'price', 'Rambience', 'franchise',
                               'area', 'other_services']]
chefmozProfile_r= chefmozProfile_r.merge(chefmozacccepts, how = 'left', on = 'placeID')
chefmozProfile_r= chefmozProfile_r.merge(chefmozcuisine, how = 'left', on = 'placeID')
chefmozProfile_r= chefmozProfile_r.merge(chefmozhours4, how = 'left', on = 'placeID')
chefmozProfile_r= chefmozProfile_r.merge(chefmozparking, how = 'left', on = 'placeID')

chefmozGeo_r = geoplaces2[['placeID', 'latitude', 'longitude', 'the_geom_meter',
                           'address', 'city', 'state', 'country', 'fax', 'zip']]

In [64]:
l,df = rs.checkData(listOfData,'?')

In [66]:
l

[(4, 'address'),
 (4, 'city'),
 (4, 'state'),
 (4, 'country'),
 (4, 'fax'),
 (4, 'zip'),
 (4, 'url'),
 (8, 'smoker'),
 (8, 'dress_preference'),
 (8, 'ambience'),
 (8, 'transport'),
 (8, 'marital_status'),
 (8, 'hijos'),
 (8, 'activity'),
 (8, 'budget')]

In [71]:
len(userprofile['userID'].unique())==len(userprofile['userID'])

True

In [72]:
len(geoplaces2['placeID'].unique())==len(geoplaces2['placeID'])

True

In [73]:
len(chefmozparking['placeID'].unique())==len(chefmozparking['placeID'])

False

In [92]:
import collections
duplicate = [item for item, count in collections.Counter(chefmozparking['placeID']).items() if count > 1]
chefmozparking[chefmozparking['placeID'].isin(duplicate)].head(5)

,placeID,parking_lot
615,132557,public
616,132557,street
619,132531,public
620,132531,street
621,132530,fee


In [93]:
len(chefmozhours4['placeID'].unique())==len(chefmozhours4['placeID'])

False

In [95]:
import collections
duplicate = [item for item, count in collections.Counter(chefmozhours4['placeID']).items() if count > 1]
chefmozhours4[chefmozhours4['placeID'].isin(duplicate)].head(5)

,placeID,hours,days
0,135111,00:00-23:30;,Mon;Tue;Wed;Thu;Fri;
1,135111,00:00-23:30;,Sat;
2,135111,00:00-23:30;,Sun;
3,135110,08:00-19:00;,Mon;Tue;Wed;Thu;Fri;
4,135110,00:00-00:00;,Sat;


In [96]:
len(chefmozcuisine['placeID'].unique())==len(chefmozcuisine['placeID'])

False

In [97]:
import collections
duplicate = [item for item, count in collections.Counter(chefmozcuisine['placeID']).items() if count > 1]
print(duplicate)
chefmozcuisine[chefmozcuisine['placeID'].isin(duplicate)].head(5)

[135103, 135102, 135101, 135100, 135099, 135098, 135097, 135092, 135091, 135086, 135073, 135071, 135053, 135052, 135043, 135041, 135032, 135030, 135029, 135026, 135020, 135007, 135004, 134997, 134990, 134983, 134982, 134976, 134972, 132962, 132917, 132867, 132866, 132852, 132851, 132774, 132773, 132555, 132531, 132530, 132527, 132521, 132493, 132491, 132483, 132474, 132449, 132443, 132442, 132439, 132419, 132417, 132413, 132392, 132341, 132333, 132329, 132328, 132319, 132318, 132317, 132312, 132302, 132296, 132289, 132284, 132273, 132258, 132255, 132253, 132246, 132244, 132243, 132240, 132238, 132237, 132235, 132228, 132226, 132223, 132217, 132216, 132214, 132207, 132187, 132186, 132184, 132182, 132177, 132171, 132166, 132161, 132155, 132147, 132137, 132128, 132126, 132125, 132109, 132108, 132101, 132100, 132031, 132030, 132028, 132024, 132023, 132012, 132010, 132005]


,placeID,Rcuisine
6,135103,Burgers
7,135103,Dessert-Ice_Cream
8,135103,Fast_Food
9,135103,Hot_Dogs
10,135102,Latin_American


In [98]:
len(chefmozacccepts['placeID'].unique())==len(chefmozacccepts['placeID'])

False

In [99]:
import collections
duplicate = [item for item, count in collections.Counter(chefmozacccepts['placeID']).items() if count > 1]
print(duplicate)
chefmozacccepts[chefmozacccepts['placeID'].isin(duplicate)].head(5)

[135110, 135107, 135106, 135104, 135103, 135102, 135099, 135098, 135097, 135093, 135091, 135089, 135086, 135084, 135081, 135080, 135079, 135078, 135076, 135074, 135073, 135072, 135071, 135068, 135067, 135066, 135065, 135064, 135061, 135058, 135057, 135055, 135054, 135053, 135052, 135050, 135048, 135047, 135046, 135045, 135040, 135037, 135035, 135034, 135033, 135032, 135031, 135029, 135028, 135027, 135025, 135024, 135018, 135008, 135007, 135006, 135004, 135002, 135001, 134994, 134993, 134992, 134991, 134989, 134988, 134985, 134984, 134983, 134978, 134977, 134976, 134975, 134974, 134973, 134972, 134971, 134970, 133043, 133033, 133030, 133027, 133026, 132958, 132957, 132956, 132955, 132954, 132953, 132947, 132931, 132927, 132917, 132900, 132892, 132888, 132887, 132886, 132880, 132875, 132873, 132872, 132863, 132862, 132861, 132860, 132859, 132857, 132854, 132835, 132826, 132824, 132820, 132811, 132808, 132807, 132806, 132804, 132802, 132801, 132799, 132797, 132796, 132795, 132793, 132792,

,placeID,Rpayment
0,135110,cash
1,135110,VISA
2,135110,MasterCard-Eurocard
3,135110,American_Express
4,135110,bank_debit_cards
